In [1]:
import requests
import os
import json
import re

# Function to extract IDs from JSON
def extract_ids_from_json(data):
    extracted_ids = []
    def search_dict(d):
        for key, value in d.items():
            if key == 'id' and isinstance(value, str):
                match = re.search(r'objects/([a-f0-9\-]+)/', value)
                if match:
                    extracted_ids.append(match.group(1))
            elif isinstance(value, dict):
                search_dict(value)
            elif isinstance(value, list):
                for item in value:
                    if isinstance(item, dict):
                        search_dict(item)
    search_dict(data)
    return extracted_ids

# Load the JSON file
with open('results.json', 'r') as f:
    data = json.load(f)

# Extract IDs
extracted_ids = extract_ids_from_json(data)

# Ensure the root directory exists
os.makedirs("my_results", exist_ok=True)

# Iterate over each extracted ID
for doc_id in extracted_ids:
    manifest_url = f"https://iiif.bodleian.ox.ac.uk/iiif/manifest/{doc_id}.json"
    response = requests.get(manifest_url)
    manifest = response.json()

    # Create a subdirectory inside "my_results"
    os.makedirs(f"my_results/{doc_id}", exist_ok=True)

    for i, canvas in enumerate(manifest["sequences"][0]["canvases"], start=1):
        image_info = canvas["images"][0]["resource"]["service"]["@id"]
        image_url = f"{image_info}/full/full/0/default.jpg"

        response = requests.get(image_url)
        # Save the image inside the subdirectory
        image_path = os.path.join(f"my_results/{doc_id}", f"page_{i}.jpg")
        with open(image_path, "wb") as file:
            file.write(response.content)

        print(f"Downloaded page {i} for document {doc_id}")

    print(f"Download completed for document {doc_id}")


FileNotFoundError: [Errno 2] No such file or directory: '/path/to/your/json/file.json'